<a href="https://colab.research.google.com/github/currencyfxjle/KNAPSACK_GDF_9.0/blob/main/KNAPSACK_GDF_9_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gspread
!pip install gspread-dataframe
!pip install --upgrade oauth2client
!pip install xlsxwriter

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import os

# List files and folders under My Drive
base_path = '/content/drive/My Drive'
for root, dirs, files in os.walk(base_path):
    print(f"Directory: {root}")
    for directory in dirs:
        print(f"  Sub-directory: {directory}")
    for file in files:
        print(f"  File: {file}")

import os

correct_path = '/content/drive/My Drive/Colab Notebooks/GDF - Knapsack/2025 Project Agenda.gsheet'

if os.path.exists(correct_path):
    print("File exists:", correct_path)
else:
    print("File does NOT exist:", correct_path)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 4.2 MB/s eta 0:00:00
Mounted at /content/drive
Directory: /content/drive/My Drive
  Sub-directory: Risk Modeling
  Sub-directory: Colab Notebooks
  File: Financial Concepts   Solar Energy Greenday Finance.gdoc
  File: config.zip
  File: 2024 Project Allocation Results (7).gsheet
  File: 2024 Project Allocation Results (6).gsheet
  File: 2024 Project Allocation Results (5).gsheet
  File: 2024 Project Allocation Results (4).gsheet
  File: 2024 Project Allocation Results (3).gsheet
  File: 2024 Project Allocation Results (2).gsheet
  File: 2024 Project Allocation Results (1).gsheet
  File: 2024 Project Allocation Results.gsheet
  File: Estados de Resultados - Victor Lizarraga.gdoc
  File: Ajustes  knapsack Lalo.gsheet
  File: Backup 2025.gsheet
  File: Ruta calculadora.gdoc
  File: KNAPSACK - GDF - Last Version 2.1.ipynb
  File: Knapsack Project.gdoc
  File: Notes.gdoc
Directory: /content/drive/My Drive/Risk Modeling
  File: Risk 

State top down and purchaser top down

In [3]:
import gspread
from google.colab import auth
from google.auth import default
import pandas as pd

# Authenticate and Access Google Sheet
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Open Google Sheet
title = "2025 Project Agenda"
sheet = gc.open(title)

# Fetch and Clean Supply Data
def fetch_supply():
    ws = sheet.worksheet("Supply")
    data = ws.get_all_values()
    supply_df = pd.DataFrame(data[1:], columns=data[0])

    # Clean and prepare data
    supply_df = supply_df[supply_df['Take into consideration?'].str.lower() == 'y']
    supply_df['Est Cash Amount'] = pd.to_numeric(supply_df['Est Cash Amount'].replace({',': ''}, regex=True))
    supply_df['Poject_date'] = pd.to_datetime(supply_df['Poject_date'], errors='coerce')

    # Convert GDF Profit to numeric
    supply_df['GDF Profit'] = supply_df['GDF Profit'].str.rstrip('%').astype(float) / 100
    supply_df = supply_df.dropna()

    # Sort projects by Est Cash Amount (descending) and date
    supply_df = supply_df.sort_values(by=['Est Cash Amount', 'Poject_date'], ascending=[False, True])

    # Add additional tracking columns
    supply_df['Original Est Cash Amount'] = supply_df['Est Cash Amount']
    supply_df['Fully Allocated'] = False

    # Add category based on Est Cash Amount
    supply_df['Project Category'] = pd.cut(
        supply_df['Est Cash Amount'],
        bins=[0, 350000, 500000, 800000, 1000000, float('inf')],
        labels=['0-350k', '350k-500k', '500k-800k', '800k-1M', '1M+']
    )

    # Include project state for information purposes
    supply_df['Project State'] = supply_df['State']

    return supply_df

# Fetch and Clean Demand Data
def fetch_demand():
    ws = sheet.worksheet("Demand")
    data = ws.get_all_values()
    demand_df = pd.DataFrame(data[1:], columns=data[0])

    # Clean and prepare data
    demand_df['approximated_appetite'] = pd.to_numeric(demand_df['approximated_appetite'].replace({',': ''}, regex=True))
    demand_df['updated pending amount'] = pd.to_numeric(demand_df['pending_amount'].replace({',': ''}, regex=True))
    demand_df['effective_date_purchaser'] = pd.to_datetime(demand_df['effective_date_purchaser'], errors='coerce')
    demand_df['total_allocated'] = 0  # Initialize tracking column
    demand_df = demand_df.dropna()

    # Add a column to track remaining appetite
    demand_df['remaining_appetite'] = demand_df['approximated_appetite']

    # Add category based on approximated appetite
    demand_df['Purchaser Category'] = pd.cut(
        demand_df['approximated_appetite'],
        bins=[0, 350000, 500000, 800000, 1000000, float('inf')],
        labels=['0-350k', '350k-500k', '500k-800k', '800k-1M', '1M+']
    )

    # Add priority for state
    demand_df['State Priority'] = demand_df['State'].apply(lambda x: 1 if x == 'CA' else 2)

    # Sort demand by approximated_appetite (descending) and state priority
    demand_df = demand_df.sort_values(by=['approximated_appetite', 'State Priority'], ascending=[False, True])

    return demand_df

# Match Supply to Demand
def match_supply_to_demand(supply_df, demand_df):
    allocations = []
    total_allocated = 0

    for idx, purchaser in demand_df.iterrows():
        remaining_pending_amount = purchaser['updated pending amount']
        remaining_appetite = purchaser['remaining_appetite']
        effective_date = purchaser['effective_date_purchaser']

        for _, project in supply_df.iterrows():
            if project['Fully Allocated']:
                continue

            project_cost = project['Est Cash Amount']
            project_date = project['Poject_date']
            gdf_profit = project['GDF Profit']  # Include GDF Profit
            project_category = project['Project Category']  # Include project category
            project_state = project['Project State']  # Include project state

            if (project_cost <= remaining_pending_amount and
                project_cost <= remaining_appetite and
                project_date >= effective_date):

                # Allocate project
                allocations.append({
                    "Purchaser Name": purchaser['purchaser_name'],
                    "Purchaser State": purchaser['State'],  # Include state
                    "Purchaser Date": effective_date,  # Include purchaser effective date
                    "Purchaser Category": purchaser['Purchaser Category'],  # Include purchaser category
                    "Project ID": project['Project ID'],
                    "Project Name": project['Proyect Name'],
                    "Project Date": project_date,  # Include project date
                    "Project Category": project_category,  # Include project category
                    "Project State": project_state,  # Include project state
                    "GDF Profit": gdf_profit,  # Include GDF Profit
                    "Allocated Amount": project_cost,
                    "Remaining Appetite After": remaining_appetite - project_cost,
                    "Updated Pending Amount After": remaining_pending_amount - project_cost,
                    "Original Appetite": purchaser['approximated_appetite']
                })

                # Update tracking
                total_allocated += project_cost
                demand_df.at[idx, 'total_allocated'] += project_cost
                remaining_pending_amount -= project_cost
                remaining_appetite -= project_cost
                supply_df.loc[project.name, 'Fully Allocated'] = True

                if remaining_pending_amount <= 0 or remaining_appetite <= 0:
                    break

        demand_df.at[idx, 'updated pending amount'] = remaining_pending_amount
        demand_df.at[idx, 'remaining_appetite'] = remaining_appetite

    return total_allocated, allocations, supply_df

# Main Execution
supply_df = fetch_supply()
demand_df = fetch_demand()

# Perform allocation
total_allocated, allocations, remaining_supply = match_supply_to_demand(supply_df, demand_df)

# Calculate totals
original_total_appetite = demand_df['approximated_appetite'].sum()
total_pending_amount = original_total_appetite - total_allocated  # Ensure accurate calculation
remaining_supply_value = remaining_supply[remaining_supply['Fully Allocated'] == False]['Est Cash Amount'].sum()

# Display results
print("\nResults Summary:")
print(f"Original Total Unallocated Value (Total Appetite): {original_total_appetite}")
print(f"Original Total Project Value (Est Cash Amount): {supply_df['Original Est Cash Amount'].sum()}")
print(f"Total Allocated: {total_allocated}")
print(f"Remaining Supply Value After: {remaining_supply_value}")
print(f"Total Pending Amount: {total_pending_amount}")

# Save allocations and summary
summary_data = pd.DataFrame([{
    "Metric": "Original Total Unallocated Value (Total Appetite)",
    "Value": original_total_appetite
}, {
    "Metric": "Original Total Project Value (Est Cash Amount)",
    "Value": supply_df['Original Est Cash Amount'].sum()
}, {
    "Metric": "Total Allocated",
    "Value": total_allocated
}, {
    "Metric": "Remaining Supply Value After",
    "Value": remaining_supply_value
}, {
    "Metric": "Total Pending Amount",
    "Value": total_pending_amount
}])

with pd.ExcelWriter("allocation_results.xlsx", engine="xlsxwriter") as writer:
    pd.DataFrame(allocations).to_excel(writer, sheet_name="Allocations", index=False)
    summary_data.to_excel(writer, sheet_name="Summary", index=False)
    print("Results saved to allocation_results.xlsx")



Results Summary:
Original Total Unallocated Value (Total Appetite): 110880429.1
Original Total Project Value (Est Cash Amount): 111770483
Total Allocated: 88138906
Remaining Supply Value After: 23631577
Total Pending Amount: 22741523.099999994
Results saved to allocation_results.xlsx


edit to interface

In [18]:
import gspread
from google.colab import auth
from google.auth import default
import pandas as pd
import ipywidgets as widgets
from IPython.display import display

# Authenticate and access Google Drive
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Function to read Google Sheet
def read_google_sheet(sheet_name_or_url):
    try:
        # Open the Google Sheet by name or URL
        if sheet_name_or_url.startswith('http'):
            sheet = gc.open_by_url(sheet_name_or_url)
        else:
            sheet = gc.open(sheet_name_or_url)

        # Access the first worksheet
        worksheet = sheet.sheet1
        data = worksheet.get_all_values()
        df = pd.DataFrame(data[1:], columns=data[0])  # First row as headers
        print("Google Sheet loaded successfully!")
        return df, worksheet
    except Exception as e:
        print(f"Error loading Google Sheet: {e}")
        return None, None

# Function to save the edited data back to the Google Sheet
def save_google_sheet(worksheet, df):
    try:
        # Clear the existing sheet and update with new data
        worksheet.clear()
        worksheet.update([df.columns.values.tolist()] + df.values.tolist())
        print("Changes saved to Google Sheet!")
    except Exception as e:
        print(f"Error saving Google Sheet: {e}")

# Function to create an editable table with filtering and editing
def create_editable_table_with_filters(df, worksheet):
    editable_df = df.copy()
    editable_table = widgets.Output()

    # Dropdown to filter by "Take into consideration?"
    status_dropdown = widgets.Dropdown(
        options=['All', 'Y', 'N'],
        description='Filter by Status:',
        disabled=False,
    )

    # Text input to filter by Project Name
    project_name_filter = widgets.Text(
        placeholder='Enter Project Name',
        description='Filter by Name:',
        disabled=False,
    )

    # Text input to filter by Project ID
    project_id_filter = widgets.Text(
        placeholder='Enter Project ID',
        description='Filter by ID:',
        disabled=False,
    )

    # Button to apply the filter
    filter_button = widgets.Button(description="Apply Filter")

    # Button to save changes
    save_button = widgets.Button(description="Save Changes")

    # Multi-select widget to select multiple projects
    project_selector = widgets.SelectMultiple(
        options=editable_df[editable_df['Take into consideration?'] == 'Y']['Project ID'].tolist(),
        description='Select Projects:',
        disabled=False,
        rows=10,  # Show 10 projects at a time
    )

    # Toggle button to change the status (Y to N or N to Y)
    toggle_button = widgets.Button(description="Toggle Status (Y/N)")

    def apply_filter(b):
        with editable_table:
            editable_table.clear_output()
            # Apply status filter
            filter_status = status_dropdown.value
            if filter_status == 'All':
                filtered_df = editable_df
            else:
                filtered_df = editable_df[editable_df['Take into consideration?'] == filter_status]

            # Apply Project Name filter
            if project_name_filter.value:
                filtered_df = filtered_df[filtered_df['Proyect Name'].str.contains(project_name_filter.value, case=False)]

            # Apply Project ID filter
            if project_id_filter.value:
                filtered_df = filtered_df[filtered_df['Project ID'].str.contains(project_id_filter.value, case=False)]

            display(filtered_df)

    def toggle_status(b):
        selected_projects = project_selector.value
        if selected_projects:
            for project_id in selected_projects:
                current_status = editable_df.loc[editable_df['Project ID'] == project_id, 'Take into consideration?'].values[0]
                new_status = 'N' if current_status == 'Y' else 'Y'
                editable_df.loc[editable_df['Project ID'] == project_id, 'Take into consideration?'] = new_status
                print(f"Project {project_id} status changed to {new_status}.")
            # Update the project selector options
            project_selector.options = editable_df[editable_df['Take into consideration?'] == 'Y']['Project ID'].tolist()
        else:
            print("No projects selected.")

    def save_changes(b):
        with editable_table:
            print("Saving changes...")
            save_google_sheet(worksheet, editable_df)
            print("Changes saved!")

    filter_button.on_click(apply_filter)
    toggle_button.on_click(toggle_status)
    save_button.on_click(save_changes)

    # Display the filter dropdowns, button, and editable table
    display(status_dropdown)
    display(project_name_filter)
    display(project_id_filter)
    display(filter_button)
    display(project_selector)
    display(toggle_button)
    display(editable_table)
    with editable_table:
        display(editable_df)
    display(save_button)

# Main Execution
if __name__ == "__main__":
    # Path to the Google Sheet (use the name or shareable URL)
    sheet_name_or_url = "2025 Project Agenda"  # Replace with the name or URL of your Google Sheet

    # Read the Google Sheet
    df, worksheet = read_google_sheet(sheet_name_or_url)

    if df is not None:
        # Ensure the "Take into consideration?" column exists
        if 'Take into consideration?' not in df.columns:
            df['Take into consideration?'] = 'Y'  # Initialize all projects as 'Y'

        # Display editable table with filtering and editing
        print("Edit Supply Data:")
        create_editable_table_with_filters(df, worksheet)

Google Sheet loaded successfully!
Edit Supply Data:


Dropdown(description='Filter by Status:', options=('All', 'Y', 'N'), value='All')

Text(value='', description='Filter by Name:', placeholder='Enter Project Name')

Text(value='', description='Filter by ID:', placeholder='Enter Project ID')

Button(description='Apply Filter', style=ButtonStyle())

SelectMultiple(description='Select Projects:', options=('Santa Cruz Nutritionals 8', 'Santa Cruz Nutritionals …

Button(description='Toggle Status (Y/N)', style=ButtonStyle())

Output()

Button(description='Save Changes', style=ButtonStyle())

No projects selected.


In [19]:
import gspread
from google.colab import auth
from google.auth import default
import pandas as pd
import ipywidgets as widgets
from IPython.display import display

# Authenticate and access Google Drive
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Function to read Google Sheet
def read_google_sheet(sheet_name_or_url):
    try:
        # Open the Google Sheet by name or URL
        if sheet_name_or_url.startswith('http'):
            sheet = gc.open_by_url(sheet_name_or_url)
        else:
            sheet = gc.open(sheet_name_or_url)

        # Access the first worksheet
        worksheet = sheet.sheet1
        data = worksheet.get_all_values()
        df = pd.DataFrame(data[1:], columns=data[0])  # First row as headers
        print("Google Sheet loaded successfully!")
        return df, worksheet
    except Exception as e:
        print(f"Error loading Google Sheet: {e}")
        return None, None

# Function to save the edited data back to the Google Sheet
def save_google_sheet(worksheet, df):
    try:
        # Clear the existing sheet and update with new data
        worksheet.clear()
        worksheet.update([df.columns.values.tolist()] + df.values.tolist())
        print("Changes saved to Google Sheet!")
    except Exception as e:
        print(f"Error saving Google Sheet: {e}")

# Function to create an editable table with filtering and editing
def create_editable_table_with_filters(df, worksheet):
    editable_df = df.copy()
    editable_table = widgets.Output()

    # Dropdown to filter by "Take into consideration?"
    status_dropdown = widgets.Dropdown(
        options=['All', 'Y', 'N'],
        description='Filter by Status:',
        disabled=False,
    )

    # Text input to filter by Project Name
    project_name_filter = widgets.Text(
        placeholder='Enter Project Name',
        description='Filter by Name:',
        disabled=False,
    )

    # Text input to filter by Project ID
    project_id_filter = widgets.Text(
        placeholder='Enter Project ID',
        description='Filter by ID:',
        disabled=False,
    )

    # Button to apply the filter
    filter_button = widgets.Button(description="Apply Filter")

    # Button to save changes
    save_button = widgets.Button(description="Save Changes")

    # Multi-select widget to select multiple projects
    project_selector = widgets.SelectMultiple(
        options=editable_df[editable_df['Take into consideration?'] == 'Y']['Project ID'].tolist(),
        description='Select Projects:',
        disabled=False,
        rows=10,  # Show 10 projects at a time
    )

    # Toggle button to change the status (Y to N or N to Y)
    toggle_button = widgets.Button(description="Toggle Status (Y/N)")

    # Button to reset all projects to Y
    reset_button = widgets.Button(description="Reset All to Y")

    def apply_filter(b):
        with editable_table:
            editable_table.clear_output()
            # Apply status filter
            filter_status = status_dropdown.value
            if filter_status == 'All':
                filtered_df = editable_df
            else:
                filtered_df = editable_df[editable_df['Take into consideration?'] == filter_status]

            # Apply Project Name filter
            if project_name_filter.value:
                filtered_df = filtered_df[filtered_df['Proyect Name'].str.contains(project_name_filter.value, case=False)]

            # Apply Project ID filter
            if project_id_filter.value:
                filtered_df = filtered_df[filtered_df['Project ID'].str.contains(project_id_filter.value, case=False)]

            display(filtered_df)

    def toggle_status(b):
        selected_projects = project_selector.value
        if selected_projects:
            for project_id in selected_projects:
                current_status = editable_df.loc[editable_df['Project ID'] == project_id, 'Take into consideration?'].values[0]
                new_status = 'N' if current_status == 'Y' else 'Y'
                editable_df.loc[editable_df['Project ID'] == project_id, 'Take into consideration?'] = new_status
                print(f"Project {project_id} status changed to {new_status}.")
            # Update the project selector options
            project_selector.options = editable_df[editable_df['Take into consideration?'] == 'Y']['Project ID'].tolist()
        else:
            print("No projects selected.")

    def reset_all_to_y(b):
        editable_df['Take into consideration?'] = 'Y'
        print("All projects reset to Y.")
        # Update the project selector options
        project_selector.options = editable_df[editable_df['Take into consideration?'] == 'Y']['Project ID'].tolist()
        with editable_table:
            editable_table.clear_output()
            display(editable_df)

    def save_changes(b):
        with editable_table:
            print("Saving changes...")
            save_google_sheet(worksheet, editable_df)
            print("Changes saved!")

    filter_button.on_click(apply_filter)
    toggle_button.on_click(toggle_status)
    reset_button.on_click(reset_all_to_y)
    save_button.on_click(save_changes)

    # Display the filter dropdowns, button, and editable table
    display(status_dropdown)
    display(project_name_filter)
    display(project_id_filter)
    display(filter_button)
    display(project_selector)
    display(toggle_button)
    display(reset_button)
    display(editable_table)
    with editable_table:
        display(editable_df)
    display(save_button)

# Main Execution
if __name__ == "__main__":
    # Path to the Google Sheet (use the name or shareable URL)
    sheet_name_or_url = "2025 Project Agenda"  # Replace with the name or URL of your Google Sheet

    # Read the Google Sheet
    df, worksheet = read_google_sheet(sheet_name_or_url)

    if df is not None:
        # Ensure the "Take into consideration?" column exists
        if 'Take into consideration?' not in df.columns:
            df['Take into consideration?'] = 'Y'  # Initialize all projects as 'Y'

        # Display editable table with filtering and editing
        print("Edit Supply Data:")
        create_editable_table_with_filters(df, worksheet)

Google Sheet loaded successfully!
Edit Supply Data:


Dropdown(description='Filter by Status:', options=('All', 'Y', 'N'), value='All')

Text(value='', description='Filter by Name:', placeholder='Enter Project Name')

Text(value='', description='Filter by ID:', placeholder='Enter Project ID')

Button(description='Apply Filter', style=ButtonStyle())

...

Button(description='Toggle Status (Y/N)', style=ButtonStyle())

Button(description='Reset All to Y', style=ButtonStyle())

Output()

Button(description='Save Changes', style=ButtonStyle())

All projects reset to Y.


Keeping track of unallocated demand & supply